In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, categories, word_to_idx, idx_to_word, embeddings = cPickle.load(open(config.path_data, 'rb'))
config.batch_size = len(instances_train)
train_batches = get_batches(instances_train, config.batch_size, iterator=False)
config.dim_bow = len(idx_to_word)
config.dim_emb = embeddings.shape[-1]

In [6]:
def debug(variable, sample_batch=None, sample=False):
    if sample_batch is None: sample_batch = train_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch, mode='eval')
    _variable = sess.run(variable, feed_dict=feed_dict)
    return _variable

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
config.train_emb = False

In [9]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [10]:
config.n_epochs = 10000
config.log_period = 500

In [11]:
time_start = time.time()
while epoch < config.n_epochs:
    ct, batch = train_batches[0]
    
    feed_dict = model.get_feed_dict(batch)
    _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
    sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

    losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
    ppls_train += list(ppls_batch)

    if global_step_log % config.log_period == 0:
        # validate
        loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
        ppl_train = np.exp(np.mean(ppls_train))
        loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, train_batches, model)

        # test
        if ppl_train < ppl_min:
            ppl_min = ppl_dev
            loss_test, _, _, _, ppl_test, _ = 0, 0, 0, 0, 0, 0
            saver.save(sess, config.path_model, global_step=global_step_log)
            cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
            update_checkpoint(config, checkpoint, global_step_log)

        # visualize topic
        topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
        topics_freq_idxs = topics_freq_indices
        topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
        topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
        descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
        recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

        depth_specs = get_topic_specialization(sess, model, instances_train)
        hierarchical_affinities = get_hierarchical_affinity(sess, model)

        # log
        clear_output()
        time_log = int(time.time() - time_start)
        log_series = pd.Series([time_log, epoch, ct, \
                '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                '%.2f'%loss_test, '%.0f'%ppl_test, \
                '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                index=log_df.columns)
        log_df.loc[global_step_log] = log_series
        display(log_df)
        cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
        print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

#         # update tree
#         if not config.static:
#             config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
#             if update_tree_flg:
#                 print(config.tree_idxs)
#                 name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
#                 if 'sess' in globals(): sess.close()
#                 model = HierarchicalNeuralTopicModel(config)
#                 sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
#                 name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
#                 sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
#                 saver = tf.train.Saver(max_to_keep=1)

        time_start = time.time()

    epoch += 1

TRAIN:                                    VALID:  \
      Time    Ep Ct       LOSS    PPL        NLL     KL   REG       LOSS   
500      9   499  0  116215.86  13812  116195.12  20.21  0.49  115238.65   
1000     6   999  0  115602.62  13383  115582.77  19.33  0.42  114799.56   
1500     7  1499  0  115308.53  13113  115288.70  19.32  0.37  114644.84   
2000     6  1999  0  115132.17  12944  115112.64  19.10  0.35  114566.43   
2500     6  2499  0  115012.51  12820  114993.40  18.89  0.34  114505.61   
3000     6  2999  0  114923.12  12721  114903.85  18.61  0.32  114453.70   
3500     7  3499  0  114853.39  12639  114834.21  18.38  0.31  114418.05   
4000     6  3999  0  114797.15  12567  114778.52  18.16  0.31  114391.66   
4500     6  4499  0  114750.93  12506  114732.82  18.02  0.30  114336.88   
5000     6  4999  0  114707.65  12453  114689.58  18.56  0.30  114287.94   
5500     6  5499  0  114670.93  12404  114649.67  18.98  0.29  114275.44   
6000     6  5999  0  114636.88  12361  114615.56  19.33  0.29  114260.34   
6500     6  6499  0  114606.56  12322  114586.68  19.67  0.28  114252.59   
7000     6  6999  0  114580.40  12288  114561.86  19.91  0.28  114235.53   
7500     7  7499  0  114557.70  12258  114537.01  20.08  0.28  114216.02   
8000     8  7999  0  114537.41  12230  114514.51  20.19  0.28  114204.43   
8500     8  8499  0  114516.87  12204  114494.59  20.24  0.27  114203.73   
9000     8  8999  0  114498.02  12181  114476.89  20.28  0.27  114192.77   
9500     8  9499  0  114481.09  12159  114461.05  20.30  0.27  114191.55   
10000    6  9999  0  114465.83  12138  114446.80  20.30  0.27  114172.30   

                                     TEST:     SPEC:             HIER:        
         PPL        NLL     KL   REG  LOSS PPL     1     2     3 CHILD OTHER  
500    13195  115218.19  20.04  0.42  0.00   0  0.67  0.46  0.38  0.58  0.44  
1000   12589  114780.04  19.23  0.30  0.00   0  0.67  0.52  0.47  0.42  0.25  
1500   12701  114625.98  18.58  0.28  0.00   0  0.66  0.52  0.48  0.42  0.24  
2000   12257  114548.11  18.04  0.28  0.00   0  0.66  0.51  0.49  0.42  0.24  
2500   12307  114487.89  17.45  0.27  0.00   0  0.66  0.51  0.48  0.42  0.24  
3000   12098  114436.30  17.14  0.26  0.00   0  0.66  0.52  0.48  0.41  0.23  
3500   12068  114400.95  16.84  0.26  0.00   0  0.66  0.52  0.48  0.41  0.23  
4000   12048  114375.20  16.19  0.26  0.00   0  0.66  0.51  0.48  0.41  0.24  
4500   12094  114316.05  20.50  0.32  0.00   0  0.66  0.37  0.38  0.70  0.55  
5000   11825  114264.38  23.32  0.24  0.00   0  0.66  0.37  0.38  0.70  0.55  
5500   11814  114251.85  23.35  0.24  0.00   0  0.66  0.37  0.38  0.69  0.55  
6000   11931  114236.02  24.09  0.24  0.00   0  0.66  0.37  0.39  0.69  0.54  
6500   11915  114229.08  23.27  0.24  0.00   0  0.66  0.37  0.39  0.68  0.54  
7000   11825  114212.60  22.69  0.24  0.00   0  0.66  0.37  0.39  0.68  0.54  
7500   11725  114193.10  22.68  0.23  0.00   0  0.66  0.37  0.40  0.66  0.53  
8000   11815  114182.70  21.49  0.23  0.00   0  0.66  0.36  0.41  0.65  0.52  
8500   12040  114182.54  20.95  0.23  0.00   0  0.66  0.36  0.42  0.64  0.51  
9000   11859  114171.99  20.54  0.23  0.00   0  0.66  0.36  0.43  0.62  0.50  
9500   11848  114170.55  20.77  0.23  0.00   0  0.66  0.36  0.43  0.62  0.50  
10000  11670  114151.90  20.17  0.23  0.00   0  0.66  0.36  0.44  0.61  0.49

0 R: 1.000 P: 0.857 cathode new_light-driven_actuators ferro-actuator pvdf_sensor water_treatment membrane_distillation capillary_microextraction solvent_/_water_media ultrafiltration_membrane all-solid-state_lithium_batteries
   1 R: 0.083 P: 0.047 vascular_grafts separator vascular_access hemodialysis_access mitral_valve_repair mf tract_reconstruction anode cathodes ptfe_grafts
     11 R: 0.010 P: 0.010 lubricants tissue_engineering ball_bearings lubricant hard_disk_lubricants biomaterial_applications tribological_applications aerospace_industry liquid_lubricant corneal_inlay
     12 R: 0.025 P: 0.025 binder electrodes electrode fuel_cell hollow_fiber_membranes fuel_cells gaskets sensor cell stent_grafts
   2 R: 0.000 P: 0.000 binder sensor sensors electrodes hollow_fiber_membranes actuators electrode lithium-ion_batteries ultrafiltration ultrafiltration_membranes
     21 R: 0.000 P: 0.000 electrodes hollow_fiber_membrane electrode binder fuel_cells hollow_fiber_membranes sensor batt